在 Python 中，数据的属性和处理方法统称属性。其实，方法是可调用的属性。
除了这二者之外，我们还可以创建特性 (prorerty)，在不该变类接口的前提下，使用存取方法修改数据属性。

# 19.1 使用动态属性转换数据

osconfeed.json 文件的记录示例；节略了部分字段的内容

In [1]:
# BEGIN OSCONFEED 下载 json 用的脚本
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # 如果需要下载，就发出提醒
        with urlopen(URL) as remote, open(JSON, 'wb') as local:  # 在 with 语句使用两个上下文管理器分别用于读取和保存远程文件
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)  # 解析 JSON 文件，返回 Python 原生对象。

# END OSCONFEED


In [3]:
feed = load()
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [4]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key)) 

  1 conferences
484 events
357 speakers
 53 venues


## 19.1.1 使用动态属性访问 JSON 类数据

我们实现了一个 FrozenJSON 类，只支持读取，即只能访问数据，不过这个类能递归，自动处理嵌套的映射和列表。

```python
# BEGIN EXPLORE0_DEMO
  >>> from osconfeed import load
  >>> raw_feed = load()
  >>> feed = FrozenJSON(raw_feed)  # 传入嵌套的字典和列表组成的 raw_feed，创建一个 FrozenJSON 实例。
  >>> len(feed.Schedule.speakers)  # FrozenJSON 实例能使用属性表示法遍历嵌套的字典；这里，我们获取列表的元素数量
  357
  >>> sorted(feed.Schedule.keys())  # 也可以使用底层字典的方法，如 keys()
  ['conferences', 'events', 'speakers', 'venues']
  >>> for key, value in sorted(feed.Schedule.items()): # 使用 items() 方法获取各个记录集合及其内容，然后显示各个记录集合中的元素数量
  ...     print('{:3} {}'.format(len(value), key))
  ...
    1 conferences
  484 events
  357 speakers
    53 venues
  >>> feed.Schedule.speakers[-1].name  # 如果内部元素是映射，会转换成 ForzenJSON 对象
  'Carina C. Zona'
  >>> talk = feed.Schedule.events[40]
  >>> type(talk) 
  <class 'explore0.FrozenJSON'>
  >>> talk.name
  'There *Will* Be Bugs'
  >>> talk.speakers  # 事件记录中有一个 speakers 列表，列出演讲者的编号
  [3471, 5199]
  >>> talk.flavor  # 读取不存在的记录会抛出 KeyError 异常
  Traceback (most recent call last):
    ...
  KeyError: 'flavor'

# END EXPLORE0_DEMO
```

FrozenJSON 类的关键是 \_\_getattr__ 方法。仅当无法使用常规方式获取属性，解释器才会调用特殊的 \_\_getattr__ 方法

In [6]:
# BEGIN EXPLORE0
from collections import abc


class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象
    """

    def __init__(self, mapping):
        self.__data = dict(mapping)  # 使用 mapping 参数构建一个字典，这么做有两个好处，一是确认传入的是字典，二是生成副本

    def __getattr__(self, name):  # 进且仅当没有指定名称的属性时才会调用 __getattr__ 方法
        if hasattr(self.__data, name):
            return getattr(self.__data, name)  # 如果 name 是实例属性 __data 的属性，返回那个属性。调用 keys 等方法就是通过这种方式处理的
        else:
            return FrozenJSON.build(self.__data[name])  # 否则，从 self.__data 获取 name 键对应的元素，返回调用 FrozenJSON.build() 方法得到的结果

    @classmethod
    def build(cls, obj):  # 这是一个备选构造方法
        if isinstance(obj, abc.Mapping):  # 如果 obj 是映射，那就构造一个 FrozenJSON 对象
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):  # 如果是 MutableSequence 对象，必然是列表，因此，我们把 obj 中每个元素递归传给 .build() 方法，构建一个列表
            return [cls.build(item) for item in obj]
        else:  # 对于其他元素，原封不动地返回
            return obj
# END EXPLORE0

注意，我们没有缓存或转换原始数据源。在迭代数据源的过程中，嵌套的数据结构不断被转换成 FrozenJSON 对象。

## 19.1.2 处理无效属性名

FrozenJSON 类有个缺陷: 没有对名称为 Python 关键字的属性做特殊处理，比如说以下情况：
```python
    >>> grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})
```
此时无法读取 grad.class 的值，因为在 Python 中 class 是保留字

当然，可以这么做:
```python
    >>> getattr(grad, 'class')
    1982
```
但是，FrozenJSON 类的目的是为了便于访问数据，因此更好的方法是检查传给 FrozenJSON.\_\_init__ 方法的映射中是否有键的名称为关键字，如果有，那么在键名后加上_，然后通过以下方式读取:
```python
    >>> grad.class_
    1982
```
因此，我们需要修改初始化方法

In [ ]:
# BEGIN EXPLORE1
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):  # keyword.iskeyword
                key += '_'
            self.__data[key] = value
# END EXPLORE1

如果 JSON 对象中的键不是有效的 Python 标识符，也会遇到类似的问题:

In [8]:
x = FrozenJSON({'2be': 'or not'})
x.2be

SyntaxError: invalid syntax (Temp/ipykernel_12976/4144259857.py, line 2)

这种有问题的键在 Python3 中易于检测，因为 str 类提供的 s.isidentifier() 方法能根据语言的语法判断 s 是否为有效的 Python 标识符。但把无效的标识符变成有效的属性名却不容易。对此，有两个简单的解决办法，一个是抛出异常，另一个是把无效的键换成通用名称。例如 attr_0、attr_1，等等。为了简单起见，我将忽略这个问题

## 19.1.3 使用 \_\_new__ 方法以灵活的方式创建对象

我们通常把 \_\_init__ 称为构造方法，这是从其他语言借鉴过来的属于。其实，用于构建实例的是特殊方法 \_\_new__ :   这是个类方法，必须返回一个实例。返回的实例会作为第一个参数(即self)传给 \_\_init__ 方法，因为调用 \_\_init__ 方法时要传入实例，而且禁止返回任何值，所以 \_\_init__ 方法其实是初始化方法。真正的构造方法是 \_\_new__ 。我们几乎不需要自己编写 \_\_new__ 方法，因为继承的实现就足够了。

但是 \_\_new__ 方法也可以返回其他类的实例，此时，解释器不会调用 \_\_init__ 方法。

Python 构建对象的过程可以用下述伪代码概括: 
```python
    # 构建对象的伪代码
    def object_maker(the_class, some_arg):
        new_object = the_class.__new__(some_arg)
        if isintance(new_object, the_class):
            the_class.__init__(new_object, some_arg)
```

In [10]:
# BEGIN EXPLORE2
from collections import abc


class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象
    """

    def __new__(cls, arg):  # 类方法，第一个参数是类本身，余下的参数与 __init__ 方法一样，只不过没有 self
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)  # 默认是行为是委托给超类的 new 方法，把唯一的参数设为 FrozenJSON
        elif isinstance(arg, abc.MutableSequence):  # 余下的代码与原先的 build 方法完全一样
            return [cls(item) for item in arg]
        else:
            return arg

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])  # 之前，这里调用的是 FrozenJSON.build 方法，现在只需调用 FrozenJSON 构造方法
# END EXPLORE2

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

##  19.1.4 使用 shelve 模块调整 OSCON 数据源的结果

shelve(架子)模块提供了pickle(泡菜)存储方式。

shelve.open 高阶函数返回一个 shelve.Shelf 实例，这是简单的键值对象数据库，背后由 dbm 模块支持，具有下述特点。
* shelve.Shelf 是 abc.MutableMapping 的子类，因此提供了处理映射类型的重要方法
* 此外，shelve.Shelf 类还提供了几个管理 I/0 的方法，如 sync 和 close；它也是一个上下文管理器
* 只要把新值赋予键，就会保存键和值
* 键必须是字符串
* 值必须是 pickle 模块能处理的对象

shelve 为识别 OSCON 的日程数据提供了以中简单有效的方式。我们将从 JSON 文件中读取所有记录，将其存在一个 shelve.Shelf 对象中，键由记录类型和编号组成，而值是我们即将定义的 Record 类的实例。

测试 schedule1 脚本
```python
# BEGIN SCHEDULE1_DEMO
    >>> import shelve
    >>> db = shelve.open(DB_NAME)  # shelve.open 函数打开现有的数据库文件，或者新建一个
    >>> if CONFERENCE not in db:  # 判断数据库是否填充的简便方法是，检查某个已知的键是否存在；这里检查的键是 conference.115，即 conference 记录的一个键
    ...     load_db(db)  # 如果数据库是空的，使用 load_db 加载数据
    ...
    >>> speaker = db['speaker.3471']  # 获取一条 speaker 数据
    >>> type(speaker)  # 打印类型为 Record 实例
    <class 'schedule1.Record'>
    >>> speaker.name, speaker.twitter  # 每个 Record 实例都有一系列自定义的属性，对应于底层 JSON 记录里的字段
    ('Anna Martelli Ravenscroft', 'annaraven')
    >>> db.close()  # 一定要记得关闭 shelve.Shelf 对象。

# END SCHEDULE1_DEMO
```

In [ ]:
# BEGIN OSCONFEED
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # <1>
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)

# END OSCONFEED

# BEGIN SCHEDULE1
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  # 这是使用关键字参数传入的属性构建实例的常用简便方式


def load_db(db):
    raw_data = load()  # 如果本地没有副本，从网上下载
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合
        record_type = collection[:-1]  # record_type 的值是去掉尾部 's' 后的集合名( events -> event )
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # 使用 record_type 和 serial 字段构成 key
            record['serial'] = key  # 把 serial 字段的值设为完整的键
            db[key] = Record(**record)  # 构建 Record 实例，存储在数据库的 key 键名下

# END SCHEDULE1

## 19.1.5 使用特性获取链接的记录

下一个版本的目标是，对于从 Shelf 对象中获取的 event 记录来说，读取它的 venue 或 speakers 属性时返回的不是编号，而是完整的记录对象。

测试 schedule2 脚本
```python
# BEGIN SCHEDULE2_DEMO

    >>> DbRecord.set_db(db)  # DbRecord 类扩展 Recode 类，添加对数据库的支持
    >>> event = DbRecord.fetch('event.33950')  # DbRecord.fetch 类方法能获取任何数据类型
    >>> event  # event 是 Event 类的实例，而 Event 类扩展 DbRecord 类
    <Event 'There *Will* Be Bugs'>
    >>> event.venue  # event.venue 返回一个 DbRecord 实例
    <DbRecord serial='venue.1449'>
    >>> event.venue.name 
    'Portland 251'
    >>> for spkr in event.speakers:  # 还可以迭代 event.speakers 列表
    ...     print('{0.serial}: {0.name}'.format(spkr))
    ...
    speaker.3471: Anna Martelli Ravenscroft
    speaker.5199: Alex Martelli

# END SCHEDULE2_DEMO
```

In [15]:
class MyType(type):
    print(123)
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)

        def inner(self):
            print('inner')
        
        cls.method_z = inner
    
    def __new__(cls, name, bases, attr_dict):
        # 创建类
        new_cls = super().__new__(cls, name, bases, attr_dict)
        return new_cls
    
    def __call__(self, *args, **kwargs):
        # 1.调用子类 new 方法创建对象
        empty_object = self.__new__(self)

        # 2.调用子类 init 初始化
        self.__init__(empty_object, *args, **kwargs)

        return empty_object

class Foo(object, metaclass = MyType):
    def __init__(self, name):
        self.name = name
    
    def __call__(self, *args, **kwargs):
        print('Foo!')

    def method_z(self):
        print('method_z')

v1 = Foo('123')
v1()
v1.method_z()

123
Foo!
inner
